<a href="https://colab.research.google.com/github/saurabhkaul/reelboost/blob/main/turboml_llm_reelboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TurboML LLM Tutorial
TurboML can spin up LLM servers with an OpenAI-compatible API. We currently support models
in the GGUF format, but also support non-GGUF models that can be converted to GGUF. In the latter
case you get to decide the quantization type you want to use.

## Set up the environment and install TurboML's SDK.
We use `turboml-installer` to set up the environment for TurboML's SDK.

In [1]:
!pip install -q turboml-installer
import turboml_installer ; turboml_installer.install_on_colab()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.6/821.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
📦 Installing...
🩹 Patching environment...
⏲ Done in 0:00:43
🔁 Restarting kernel...


The kernel should now be restarted with TurboML's SDK installed.

## Login to your TurboML instance

Note that you can copy and replace this snippet with one from your TurboML homepage.

In [3]:
import turboml as tb

tb.init(
  backend_url="https://screeching-dolphin.api.turboml.online",
  api_key="tb_iVKKijh8TKeezNjButxCsCHqdYi8HreO_7e07ce66"
)

In [4]:
LlamaServerRequest = tb.llm.LlamaServerRequest
HuggingFaceSpec = LlamaServerRequest.HuggingFaceSpec
ServerParams = LlamaServerRequest.ServerParams

## Choose a model
Let's use a Llama 3.2 quant already in the GGUF format.

In [7]:
hf_spec = HuggingFaceSpec(
    hf_repo_id="Mozilla/llava-v1.5-7b-llamafile",
    select_gguf_file="llava-v1.5-7b-Q4_K.gguf",
)

## Spawn a server
On spawning a server, you get a `server_id` to reference it later as well as `server_relative_url` you can
use to reach it. This method is synchronous, so it can take a while to yield as we retrieve (and convert) your model.

In [8]:
response = tb.llm.spawn_llm_server(
    LlamaServerRequest(
        source_type=LlamaServerRequest.SourceType.HUGGINGFACE,
        hf_spec=hf_spec,
        server_params=ServerParams(
            threads=-1,
            seed=-1,
            context_size=0,
            flash_attention=False,
        ),
    )
)
response

INFO:turboml.llm:[hf-acquisition] Status: in_progress, Progress: Downloading model from HF...
INFO:turboml.llm:[hf-acquisition] Status: completed, Progress: Completed successfully.
INFO:turboml.llm:[hf-acquisition] Acquisition Done, gguf_id = Mozilla$$llava-v1.5-7b-llamafile$$llava-v15-7b-Q4_Kgguf


LlamaServerResponse(server_id='Mozilla$$llava-v1.5-7b-llamafile$$llava-v15-7b-Q4_Kgguf.1939615351', server_relative_url='/openai/Mozilla$$llava-v1.5-7b-llamafile$$llava-v15-7b-Q4_Kgguf.1939615351/api/v1')

In [9]:
server_id = response.server_id

In [ ]:
from IPython.display import display, Image, Audio
import cv2  # We're using OpenCV to read video, to install !pip install opencv-python
import base64
import time
# from openai import OpenAI
import os
import requests
import datetime

### Interacting with the LLM

Our LLM is exposed with an OpenAI-compatible API, so we can use the OpenAI SDK, or any
other tool compatible tool to use it.

In [ ]:
from openai import OpenAI

base_url = tb.common.env.CONFIG.TURBOML_BACKEND_SERVER_ADDRESS
server_url = f"{base_url}/{response.server_relative_url}"

client = OpenAI(base_url=server_url, api_key="-")


response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello there how are you doing today?",
        }
    ],
    model="-",
)

print(response)

In [10]:
%pip install openai

In [2]:
embeddings = (
    client.embeddings.create(input=["Hello there how are you doing today?"], model="-")
    .data[0]
    .embedding
)
len(embeddings), embeddings[:5]

NameError: name 'client' is not defined

## Stop the server

In [ ]:
tb.llm.stop_llm_server(server_id)